In [1]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


______________________________________________________________________________________________**

In [ ]:
import tkinter as tk
import json
from tkinter import filedialog, simpledialog, messagebox
from PIL import Image, ImageTk
from PIL import ImageGrab
from PIL import ImageDraw


class MeasureTool:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Measuring Tool")

        self.root.geometry('1200x800')

        self.line_mode = False

        self.menu = tk.Menu(self.root)
        self.root.config(menu=self.menu)
        self.file_menu = tk.Menu(self.menu)
        self.menu.add_cascade(label="File", menu=self.file_menu)
        self.file_menu.add_command(label="Load Image", command=self.load_image)
        self.menu.add_command(label="Debug", command=self.debug_output)

        self.canvas_frame = tk.Frame(self.root)
        self.controls_frame = tk.Frame(self.root, width=200)
        self.controls_frame.pack_propagate(0) 

        self.v_scroll = tk.Scrollbar(self.canvas_frame, orient=tk.VERTICAL)
        self.v_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        self.h_scroll = tk.Scrollbar(self.canvas_frame, orient=tk.HORIZONTAL)
        self.h_scroll.pack(side=tk.BOTTOM, fill=tk.X)
        self.canvas = tk.Canvas(self.canvas_frame, bg="white", yscrollcommand=self.v_scroll.set, xscrollcommand=self.h_scroll.set)
        self.canvas.pack(fill=tk.BOTH, expand=True)
        self.v_scroll.config(command=self.canvas.yview)
        self.h_scroll.config(command=self.canvas.xview)

        self.chain1_btn = tk.Button(self.controls_frame, text="Add Chain 1", command=self.add_chain1)
        self.chain1_btn.pack(fill=tk.X)
        self.clear_chain1_btn = tk.Button(self.controls_frame, text="Clear Chain 1", command=self.clear_chain1)
        self.clear_chain1_btn.pack(fill=tk.X)
        self.chain2_btn = tk.Button(self.controls_frame, text="Add Chain 2", command=self.add_chain2)
        self.chain2_btn.pack(fill=tk.X)
        self.clear_chain2_btn = tk.Button(self.controls_frame, text="Clear Chain 2", command=self.clear_chain2)
        self.clear_chain2_btn.pack(fill=tk.X)
        self.add_lines_btn = tk.Button(self.controls_frame, text="Add Lines", command=self.add_lines_mode)
        self.add_lines_btn.pack(pady=10, fill=tk.X)
        self.calculate_btn = tk.Button(self.controls_frame, text="Calculate Distance", command=self.calculate_distance)
        self.calculate_btn.pack(pady=10, fill=tk.X)

        self.raster_label = tk.Label(self.controls_frame, text="Rasterization:")
        self.raster_label.pack(side=tk.TOP, anchor="w", padx=5)
        self.raster_entry = tk.Entry(self.controls_frame, width=10)
        self.raster_entry.insert(0, "1") 
        self.raster_entry.pack(side=tk.TOP, anchor="w", padx=5)

        self.scale_label = tk.Label(self.controls_frame, text="Scaling Factor:")
        self.scale_label.pack(side=tk.TOP, anchor="w", padx=5)
        self.scale_entry = tk.Entry(self.controls_frame, width=10)
        self.scale_entry.insert(0, "1") 
        self.scale_entry.pack(side=tk.TOP, anchor="w", padx=5)

        self.line1_distance_label = tk.Label(self.controls_frame, text="Line 1 Distance:")
        self.line1_distance_label.pack(side=tk.TOP, anchor="w", padx=5)
        self.line1_distance_entry = tk.Entry(self.controls_frame, width=5)
        self.line1_distance_entry.insert(0, "76") 
        self.line1_distance_entry.pack(side=tk.TOP, anchor="w", padx=5)

        self.line2_distance_label = tk.Label(self.controls_frame, text="Line 2 Distance:")
        self.line2_distance_label.pack(side=tk.TOP, anchor="w", padx=5)
        self.line2_distance_entry = tk.Entry(self.controls_frame, width=5)
        self.line2_distance_entry.insert(0, "695")
        self.line2_distance_entry.pack(side=tk.TOP, anchor="w", padx=5)

        self.line3_distance_label = tk.Label(self.controls_frame, text="Line 3 Distance:")
        self.line3_distance_label.pack(side=tk.TOP, anchor="w", padx=5)
        self.line3_distance_entry = tk.Entry(self.controls_frame, width=5)
        self.line3_distance_entry.insert(0, "1110") 
        self.line3_distance_entry.pack(side=tk.TOP, anchor="w", padx=5)

        self.measurement_interval_label = tk.Label(self.controls_frame, text="Disp. Measure Interval:")
        self.measurement_interval_label.pack(side=tk.TOP, anchor="w", padx=5)
        self.measurement_interval_entry = tk.Entry(self.controls_frame)
        self.measurement_interval_entry.pack(side=tk.TOP, anchor="w", padx=5)
        self.measurement_interval_entry.insert(0, "100")

        self.orientation_var = tk.StringVar(self.root)
        self.orientation_var.set("Vertical") 
        self.orientation_menu = tk.OptionMenu(self.controls_frame, self.orientation_var, "Vertical", "Horizontal")
        self.orientation_menu.pack(pady=10, padx=20, side=tk.TOP, anchor="w")

        self.result_text = tk.Text(self.controls_frame, width=220, height=500)
        self.result_text.pack(pady=20, side=tk.TOP, fill=tk.BOTH)
        
        self.root.bind("<Left>", self.scroll_left)
        self.root.bind("<Right>", self.scroll_right)
        self.root.bind("<Up>", self.scroll_up)
        self.root.bind("<Down>", self.scroll_down)
        
        self.root.bind("<Control-plus>", self.zoom_in)
        self.root.bind("<Control-minus>", self.zoom_out)


        self.chain1 = []
        self.chain2 = []
        self.active_chain = None
        self.image = None
        self.lines_added = []

        self.drawn_items = []
        self.drawn_chains = []
        self.measurement_lines = []
        self.measurement_texts = []

        self.canvas.bind("<Button-1>", self.add_point)

        platform = self.root.tk.call('tk', 'windowingsystem')
        if platform == 'win32':
            self.canvas.bind("<MouseWheel>", self.scroll_vertical)
        else: 
            self.canvas.bind("<Button-4>", self.scroll_vertical)
            self.canvas.bind("<Button-5>", self.scroll_vertical)
        self.canvas.bind("<Control-MouseWheel>", self.scroll_horizontal)
        if platform != 'win32': 
            self.canvas.bind("<Control-Button-4>", self.scroll_horizontal)
            self.canvas.bind("<Control-Button-5>", self.scroll_horizontal)
        self.canvas.bind("<Shift-MouseWheel>", self.zoom)
        if platform != 'win32': 
            self.canvas.bind("<Shift-Button-4>", self.zoom)
            self.canvas.bind("<Shift-Button-5>", self.zoom)

        self.original_image = None
        self.scaled_image = None
        self.scale_factor = 1.0

        self.file_menu.add_command(label="1. Save Data", command=self.save_data)
        self.file_menu.add_command(label="2. Save Canvas as Image", command=self.save_canvas_image)
        self.file_menu.add_command(label="Load Data", command=self.load_data)
        
        self.root.grid_rowconfigure(0, weight=1) 
        self.root.grid_columnconfigure(0, weight=1) 
        self.root.grid_columnconfigure(1, minsize=200) 

        self.canvas_frame.grid(row=0, column=0, sticky="nsew")
        self.controls_frame.grid(row=0, column=1, sticky="nsew") 
        
    def scroll_left(self, event):
        self.canvas.xview_scroll(-1, "units")

    def scroll_right(self, event):
        self.canvas.xview_scroll(1, "units")

    def scroll_up(self, event):
        self.canvas.yview_scroll(-1, "units")

    def scroll_down(self, event):
        self.canvas.yview_scroll(1, "units")
        
    def zoom_in(self, event):
        zoom_event = tk.Event()
        zoom_event.delta = 120
        self.zoom(zoom_event)

    def zoom_out(self, event):
        zoom_event = tk.Event()
        zoom_event.delta = -120
        self.zoom(zoom_event)

      
    def save_data(self):
        file_path = filedialog.asksaveasfilename(title="Save Data", defaultextension=".json", filetypes=[("JSON files", "*.json"), ("All files", "*.*")])
        if not file_path:
            return

        measurements_lines = self.result_text.get(1.0, tk.END).strip().split("\n")
        measurements_csv = ",".join(measurements_lines)

        data = {
            'chain1': self.chain1,
            'chain2': self.chain2,
            'image_path': self.image_path if hasattr(self, 'image_path') else None,
            'scale_factor': self.scale_factor,
            'measurements_csv': measurements_csv,
            'lines_added': self.lines_added
        }

        with open(file_path, 'w') as file:
            json.dump(data, file)

            
    def add_lines_mode(self):
        self.line_mode = True
            
    def load_data(self):
        self.reset_app()
        file_path = filedialog.askopenfilename(title="Load Data", defaultextension=".json", filetypes=[("JSON files", "*.json"), ("All files", "*.*")])
        if not file_path:
            return

        with open(file_path, 'r') as file:
            data = json.load(file)

        if 'measurements_csv' in data:
            measurements_lines = data['measurements_csv'].split(",")
            measurements_multiline = "\n".join(measurements_lines)
            self.result_text.delete(1.0, tk.END)
            self.result_text.insert(tk.END, measurements_multiline)

        self.scale_factor = 1.0

        self.image_path = data['image_path']
        self.image = Image.open(self.image_path)
        self.original_image = self.image.copy()

        self.scale_factor = data['scale_factor']
        new_width = int(self.original_image.width * self.scale_factor)
        new_height = int(self.original_image.height * self.scale_factor)
        self.scaled_image = self.original_image.resize((new_width, new_height))
        self.tk_image = ImageTk.PhotoImage(self.scaled_image)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)

        self.canvas.config(width=new_width, height=new_height)
        self.canvas.configure(scrollregion=self.canvas.bbox(tk.ALL))

        self.chain1 = data['chain1']
        self.chain2 = data['chain2']

        if 'lines_added' in data:
            self.lines_added = data['lines_added']
            for line_dict in self.lines_added:
                line_type = line_dict.get('type')
                if line_type == 'vertical':
                    x = line_dict['x']
                    for i in range(len(line_dict['y_positions']) - 1):
                        y1 = line_dict['y_positions'][i]
                        y2 = line_dict['y_positions'][i + 1]
                        self.canvas.create_line(x, y1, x, y2, fill="red", width=2)
                elif line_type == 'horizontal':
                    y = line_dict['y']
                    for i in range(len(line_dict['x_positions']) - 1):
                        x1 = line_dict['x_positions'][i]
                        x2 = line_dict['x_positions'][i + 1]
                        self.canvas.create_line(x1, y, x2, y, fill="red", width=2)
                        
        self.redraw_canvas()

        
    def load_image(self):
        file_path = filedialog.askopenfilename(title="Select an Image", filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")])
        if not file_path:
            return

        self.image_path = file_path

        self.image = Image.open(file_path)

        self.scale_factor = 0.5

        new_width = int(self.image.width * self.scale_factor)
        new_height = int(self.image.height * self.scale_factor)
        self.scaled_image = self.image.resize((new_width, new_height))
        self.tk_image = ImageTk.PhotoImage(self.scaled_image)

        self.canvas.config(width=new_width, height=new_height)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)

        extra_width = 210
        extra_height = 20
        self.root.geometry(f"{new_width + extra_width}x{new_height + extra_height}")

        self.chain1.clear()
        self.chain2.clear()

        self.original_image = self.image.copy()


    def add_chain1(self):
        self.active_chain = self.chain1
        self.chain1_btn.config(state=tk.DISABLED)
        self.chain2_btn.config(state=tk.NORMAL)

    def add_chain2(self):
        self.active_chain = self.chain2
        self.chain1_btn.config(state=tk.NORMAL)
        self.chain2_btn.config(state=tk.DISABLED)

    def add_point(self, event):
        scaling_factor = float(self.get_scaling_factor())
       
        abs_x = self.canvas.canvasx(event.x)
        abs_y = self.canvas.canvasy(event.y)
        
        if self.line_mode:
            x, y = abs_x / self.scale_factor, abs_y / self.scale_factor
            img_width = self.original_image.width * self.scale_factor
            img_height = self.original_image.height * self.scale_factor

            distances = [
                0,
                int(self.line1_distance_entry.get()),
                int(self.line2_distance_entry.get()),
                int(self.line3_distance_entry.get())
            ]

            if self.orientation_var.get() == "Vertical":
                vertical_line = self.canvas.create_line(x * self.scale_factor, 0, x * self.scale_factor, img_height, fill="red")
                self.drawn_items.append(vertical_line)

                y_positions = [y  * self.scale_factor - d * self.scale_factor * scaling_factor for d in distances]
                for y_pos in y_positions:
                    horizontal_line = self.canvas.create_line(0, y_pos , img_width , y_pos , fill="blue")
                    self.drawn_items.append(horizontal_line)

                y_positions = [y - d  for d in distances]
                self.lines_added.append({
                    'type': 'vertical',
                    'x': x,
                    'y_positions': y_positions
                })

            else: 
                horizontal_line = self.canvas.create_line(0, y * self.scale_factor, img_width, y * self.scale_factor, fill="red")
                self.drawn_items.append(horizontal_line)

                x_positions = [x  * self.scale_factor + d * self.scale_factor* scaling_factor for d in distances]
                for x_pos in x_positions:
                    vertical_line = self.canvas.create_line(x_pos , 0, x_pos , img_height, fill="blue")
                    self.drawn_items.append(vertical_line)

                x_positions = [x  + d for d in distances]
                self.lines_added.append({
                    'type': 'horizontal',
                    'y': y,
                    'x_positions': x_positions
                })

            self.line_mode = False

        else:
            if self.active_chain is None:
                return

            x, y = abs_x / self.scale_factor, abs_y / self.scale_factor
            color = "orange" if self.active_chain == self.chain1 else "green"

            scaled_x, scaled_y = x * self.scale_factor, y * self.scale_factor

            oval = self.canvas.create_oval(scaled_x-2, scaled_y-2, scaled_x+2, scaled_y+2, fill=color, outline=color)
            self.drawn_items.append(oval)

            if self.active_chain:
                last_x, last_y = self.active_chain[-1]
                last_x, last_y = last_x * self.scale_factor, last_y * self.scale_factor
                line = self.canvas.create_line(last_x, last_y, scaled_x, scaled_y, fill=color)
                self.drawn_items.append(line)

            self.active_chain.append((x, y))


        
    def interpolate(self, x, x1, y1, x2, y2):

        return y1 + (x - x1) * (y2 - y1) / (x2 - x1)
    
    def get_y_at_x(self, x, chain):

        for i in range(1, len(chain)):
            x1, y1 = chain[i-1]
            x2, y2 = chain[i]
            if x1 <= x <= x2:
                return self.interpolate(x, x1, y1, x2, y2)
        return None
    
    def calculate_distance(self):


        self.clear_measurement_lines()
        self.result_text.delete(1.0, tk.END)        
        raster_value = self.raster_entry.get() 
        if not raster_value.replace(".", "", 1).isdigit():
            messagebox.showerror("Error", "Please enter a valid rasterization value!")
            return
        scaling_factor = float(self.get_scaling_factor())
        raster_value = float(raster_value) * scaling_factor
        
        measurement_interval = self.measurement_interval_entry.get()
        if not measurement_interval.isdigit():
            messagebox.showerror("Error", "Please enter a valid measurement interval value!")
            return
        measurement_interval = float(measurement_interval) * scaling_factor
        
        if not self.chain1 or not self.chain2:
            messagebox.showerror("Error", "Both chains should have points!")
            return

        self.result_text.insert(tk.END, "Distance Along Orange\tOrthogonal Distance to Green\n")
        self.result_text.insert(tk.END, "----------------------\t-----------------------------\n")

        total_distance = 0
        next_measurement_distance = 0

        for i in range(1, len(self.chain1)):
                point1 = self.chain1[i - 1]
                point2 = self.chain1[i]
                dx, dy = point2[0] - point1[0], point2[1] - point1[1]
                ortho_dx, ortho_dy = dy, -dx

                segment_length = ((dx ** 2 + dy ** 2) ** 0.5)

                num_points = int(segment_length / raster_value)
                for j in range(num_points):
                    fraction = j / num_points
                    x = point1[0] + fraction * dx
                    y = point1[1] + fraction * dy

                    intersection = self.get_intersection((x, y), (ortho_dx, ortho_dy), self.chain2)

                    if intersection:
                        intersect_x, intersect_y = intersection
                        orthogonal_distance = (((intersect_x - x) ** 2 + (intersect_y - y) ** 2) ** 0.5) * scaling_factor
                        total_distance += raster_value
                        self.result_text.insert(tk.END, f"{total_distance:.2f}\t\t{orthogonal_distance:.2f}\n")
                        
                        if total_distance == raster_value or abs(total_distance % measurement_interval) < raster_value:
                            scaled_x, scaled_y = x * self.scale_factor, y * self.scale_factor
                            scaled_intersect_x, scaled_intersect_y = intersect_x * self.scale_factor, intersect_y * self.scale_factor
                            line = self.canvas.create_line(scaled_x, scaled_y, scaled_intersect_x, scaled_intersect_y, fill="purple", dash=(2, 2))
                            self.measurement_lines.append(line)

                            mid_x = (scaled_x + scaled_intersect_x) / 2
                            mid_y = (scaled_y + scaled_intersect_y) / 2
                            distance_str = f"{orthogonal_distance:.2f}"
                            rect = self.canvas.create_rectangle(0, 0, 0, 0, fill="black")  
                            text = self.canvas.create_text(mid_x, mid_y, text=distance_str, fill="white", font=("Arial", 10, "bold"))
                            self.canvas.coords(rect, self.canvas.bbox(text)) 
                            self.canvas.tag_lower(rect, text)  
                            self.measurement_texts.append((rect, text))

    
    def zoom(self, event):
        scaling_factor = float(self.get_scaling_factor())
        platform = self.root.tk.call('tk', 'windowingsystem')

        if platform == 'win32':
            factor = 1.1 if event.delta > 0 else 0.9
        else:
            factor = 1.1 if event.num == 4 else 0.9

        old_scale_factor = self.scale_factor
        self.scale_factor *= factor

        new_width = int(self.original_image.width * self.scale_factor)
        new_height = int(self.original_image.height * self.scale_factor)

        self.canvas.delete(tk.ALL)

        self.scaled_image = self.original_image.resize((new_width, new_height))
        self.tk_image = ImageTk.PhotoImage(self.scaled_image)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)

        scaled_chain1 = [(int(point[0] * self.scale_factor), int(point[1] * self.scale_factor)) for point in self.chain1]
        scaled_chain2 = [(int(point[0] * self.scale_factor), int(point[1] * self.scale_factor)) for point in self.chain2]
        self.draw_chain(scaled_chain1, "orange")
        self.draw_chain(scaled_chain2, "green")

        for line_info in self.lines_added:
            if line_info['type'] == 'vertical':
                x = line_info['x'] * self.scale_factor
                y_positions = [y_pos * self.scale_factor for y_pos in line_info['y_positions']]
                vertical_line = self.canvas.create_line(x, 0, x, new_height, fill="red")
                self.drawn_items.append(vertical_line)
                for y_pos in y_positions:
                    horizontal_line = self.canvas.create_line(0, y_pos, new_width, y_pos, fill="blue")
                    self.drawn_items.append(horizontal_line)
            else:
                y = line_info['y'] * self.scale_factor
                x_positions = [x_pos * self.scale_factor for x_pos in line_info['x_positions']]
                horizontal_line = self.canvas.create_line(0, y, new_width, y, fill="red")
                self.drawn_items.append(horizontal_line)
                for x_pos in x_positions:
                    vertical_line = self.canvas.create_line(x_pos, 0, x_pos, new_height, fill="blue")
                    self.drawn_items.append(vertical_line)

        canvas_width = min(self.root.winfo_width() - 210, new_width)
        canvas_height = min(self.root.winfo_height() - 310, new_height)
        self.canvas.config(width=canvas_width, height=canvas_height)
        self.canvas.configure(scrollregion=self.canvas.bbox(tk.ALL))

    def draw_chain(self, chain, color):
        point_radius = 2

        previous_point = None
        for point in chain:
            x, y = point
            
            oval = self.canvas.create_oval(x - point_radius, y - point_radius, 
                                           x + point_radius, y + point_radius, 
                                           fill=color, outline=color)
            self.drawn_chains.append(oval)
            
            if previous_point:
                line = self.canvas.create_line(previous_point[0], previous_point[1], x, y, fill=color)
                self.drawn_chains.append(line)

            previous_point = point

                
                
    def get_intersection(self, point, direction, chain):
        x, y = point
        dx, dy = direction

        scalar = 1000

        x1, y1 = x - scalar * dx, y - scalar * dy
        x2, y2 = x + scalar * dx, y + scalar * dy

        for i in range(1, len(chain)):
            seg_x1, seg_y1 = chain[i-1]
            seg_x2, seg_y2 = chain[i]
            denom = (x1 - x2) * (seg_y1 - seg_y2) - (y1 - y2) * (seg_x1 - seg_x2)
            if denom == 0:
                continue

            t = ((x1 - seg_x1) * (seg_y1 - seg_y2) - (y1 - seg_y1) * (seg_x1 - seg_x2)) / denom
            u = -((x1 - x2) * (y1 - seg_y1) - (y1 - y2) * (x1 - seg_x1)) / denom

            if 0 <= t <= 1 and 0 <= u <= 1:
                intersect_x = x1 + t * (x2 - x1)
                intersect_y = y1 + t * (y2 - y1)
                return intersect_x, intersect_y

        return None
    
    def cumulative_distance(self, chain, index):
        distance = 0
        for i in range(1, index+1):
            x1, y1 = chain[i-1]
            x2, y2 = chain[i]
            distance += ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
        return distance

    def clear_chain1(self):
        self.chain1.clear()
        self.redraw_canvas()
        self.chain1_btn.config(state=tk.NORMAL)

    def clear_chain2(self):
        self.chain2.clear()
        self.redraw_canvas()
        self.chain2_btn.config(state=tk.NORMAL)

    def redraw_canvas(self):
        self.canvas.delete(tk.ALL)
        if self.tk_image:
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)

        for x, y in self.chain1:
            x = x * self.scale_factor
            y = y * self.scale_factor
            self.canvas.create_oval(x-2, y-2, x+2, y+2, fill="orange", outline="orange")

        for i in range(1, len(self.chain1)):
            x1, y1 = self.chain1[i-1]
            x2, y2 = self.chain1[i]
            self.canvas.create_line(x1 * self.scale_factor, y1 * self.scale_factor, x2 * self.scale_factor, y2 * self.scale_factor, fill="orange")

        for x, y in self.chain2:
            x = x * self.scale_factor
            y = y * self.scale_factor
            self.canvas.create_oval(x-2, y-2, x+2, y+2, fill="green", outline="green")

        for i in range(1, len(self.chain2)):
            x1, y1 = self.chain2[i-1]
            x2, y2 = self.chain2[i]
            self.canvas.create_line(x1 * self.scale_factor, y1 * self.scale_factor, x2 * self.scale_factor, y2 * self.scale_factor, fill="green")

        for item in self.lines_added:
            if item['type'] == 'vertical':
                x = item['x'] * self.scale_factor
                vertical_line = self.canvas.create_line(x, 0, x, self.scaled_image.height, fill="red")
                y_positions = [y_pos * self.scale_factor for y_pos in item['y_positions']]
                for y_pos in y_positions:
                    horizontal_line = self.canvas.create_line(0, y_pos, self.scaled_image.width, y_pos, fill="blue")
            else:
                y = item['y'] * self.scale_factor
                horizontal_line = self.canvas.create_line(0, y, self.scaled_image.width, y, fill="red")
                x_positions = [x_pos * self.scale_factor for x_pos in item['x_positions']]
                for x_pos in x_positions:
                    vertical_line = self.canvas.create_line(x_pos, 0, x_pos, self.scaled_image.height, fill="blue")
                
    def save_canvas_image(self):
        self.canvas.delete(tk.ALL)
        if self.tk_image:
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)
            
        img_to_save = self.original_image.copy()
        draw = ImageDraw.Draw(img_to_save)

        for i in range(1, len(self.chain1)):
            x1, y1 = (coord for coord in self.chain1[i-1])
            x2, y2 = (coord for coord in self.chain1[i])
            draw.line([(x1, y1), (x2, y2)], fill="orange", width=2)
            draw.ellipse([(x1-2, y1-2), (x1+2, y1+2)], fill="orange", outline="orange")

        for i in range(1, len(self.chain2)):
            x1, y1 = (coord for coord in self.chain2[i-1])
            x2, y2 = (coord for coord in self.chain2[i])
            draw.line([(x1, y1), (x2, y2)], fill="green", width=2)
            draw.ellipse([(x1-2, y1-2), (x1+2, y1+2)], fill="green", outline="green")

        for line_info in self.lines_added:
            if line_info['type'] == 'vertical':
                x = line_info['x']
                y_positions = line_info['y_positions']
                draw.line([(x, 0), (x, self.original_image.height)], fill="red", width=2)
                for y_pos in y_positions:
                    draw.line([(0, y_pos), (self.original_image.width, y_pos)], fill="blue", width=2)
            else:
                y = line_info['y']
                x_positions = line_info['x_positions']
                draw.line([(0, y), (self.original_image.width, y)], fill="red", width=2)
                for x_pos in x_positions:
                    draw.line([(x_pos, 0), (x_pos, self.original_image.height)], fill="blue", width=2)

        file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("JPEG files", "*.jpg"), ("All files", "*.*")])

        if file_path:
            img_to_save.save(file_path)
            
        self.redraw_canvas()
            
    def reset_app(self):
        self.chain1 = []
        self.chain2 = []
        self.active_chain = None

        self.image = None
        self.original_image = None
        self.scaled_image = None
        self.image_path = None

        self.drawn_items = []
        self.scale_factor = 1.0

        self.canvas.delete(tk.ALL)

        self.chain1_btn.config(state=tk.NORMAL)
        self.chain2_btn.config(state=tk.NORMAL)
        
    def scroll_vertical(self, event):
        """Scroll canvas vertically using the mouse wheel."""
        if event.num == 4 or event.delta > 0:
            self.canvas.yview_scroll(-1, "units")
        elif event.num == 5 or event.delta < 0:
            self.canvas.yview_scroll(1, "units")

    def scroll_horizontal(self, event):
        """Scroll canvas horizontally using Ctrl + mouse wheel."""
        if event.num == 4 or event.delta > 0:
            self.canvas.xview_scroll(-1, "units")
        elif event.num == 5 or event.delta < 0:
            self.canvas.xview_scroll(1, "units")
            
    def get_scaling_factor(self):
        try:
            return float(self.scale_entry.get())
        except ValueError:
            messagebox.showerror("Error", "Invalid scaling factor! Please enter a valid number.")
            return None
            
    def debug_output(self):
        print("Chain 1 Coordinates:")
        for x, y in self.chain1:
            print(f"({x}, {y})")

        print("\nChain 2 Coordinates:")
        for x, y in self.chain2:
            print(f"({x}, {y})")

        print("\nLines Added:")
        for line in self.lines_added:
            print(line) 
            
    def clear_drawn_items(self):
        for item in self.drawn_items:
            self.canvas.delete(item)
        self.drawn_items.clear()
    
    def clear_measurement_lines(self):
        for line in self.measurement_lines:
            self.canvas.delete(line)
        self.measurement_lines.clear()
        
        for rect, text in self.measurement_texts:
            self.canvas.delete(rect)
            self.canvas.delete(text)
        self.measurement_texts.clear()



root = tk.Tk()
tool = MeasureTool(root)
root.mainloop()

